In [24]:
import json
import time

import numpy as np
import scipy.sparse as sp
from problems import Problem_sparse
from bounds.primary.truth import truth
from problems import solve
import seaborn as sns
import os
from mip import Model

In [51]:
folder = "/Users/gderval/Downloads/Data-Netlib-master/"
files = [x for x in os.listdir(folder) if x.endswith(".mps")]

files.remove("dfl001.mps")
files.remove("e226.mps")
files.remove("afiro.mps")
files.remove("sc50b.mps")
files.remove("sc50a.mps")
#files.remove("pilot.mps")
#files.remove("pilot87.mps")
files.remove("kb2.mps")
files.remove("vtpbase.mps")
files.remove("share2b.mps")
files.remove("scagr7.mps")
files.remove("forplan.mps")
files.remove("adlittle.mps")
files.remove("scsd1.mps")
files = files[files.index("agg2.mps")+1:]
len(files), files

(56,
 ['lotfi.mps',
  'agg3.mps',
  'ship04l.mps',
  'ship12l.mps',
  'scagr25.mps',
  'fit2d.mps',
  'sctap2.mps',
  'greenbeb.mps',
  'scsd6.mps',
  'sctap3.mps',
  'wood1p.mps',
  'd6cube.mps',
  'sctap1.mps',
  'tuff.mps',
  'greenbea.mps',
  'shell.mps',
  'd2q06c.mps',
  'perold.mps',
  'gfrd-pnc.mps',
  'ship08s.mps',
  'scsd8.mps',
  'fffff800.mps',
  'seba.mps',
  'grow7.mps',
  'grow22.mps',
  'cycle.mps',
  'brandy.mps',
  'ship12s.mps',
  'ship04s.mps',
  'finnis.mps',
  'stocfor1.mps',
  'recipe.mps',
  'scfxm3.mps',
  'scfxm2.mps',
  'agg.mps',
  'fit1d.mps',
  'stocfor2.mps',
  'sc105.mps',
  'scorpion.mps',
  'czprob.mps',
  'scfxm1.mps',
  'fit1p.mps',
  'pilot4.mps',
  'blend.mps',
  'degen2.mps',
  'degen3.mps',
  'share1b.mps',
  'woodw.mps',
  'capri.mps',
  'modszk1.mps',
  'sierra.mps',
  'israel.mps',
  'boeing1.mps',
  '80bau3b.mps',
  'boeing2.mps',
  'sc205.mps'])

In [54]:
from bounds.primary.robust import robust_concave_envelope

MODE = "EQ"

def gen_random(filename):
    mps = Model()
    mps.read(folder+filename)
    
    a_eq = []
    b_eq = []
    a_ineq = []
    b_ineq = []
    
    for ctr in mps.constrs:
        mul = -1 if ctr.expr.sense == ">" else 1
        if ctr.expr.sense == "=":
            a = a_eq
            b = b_eq
        else:
            a = a_ineq
            b = b_ineq
        ctr_idx = len(b)
        
        for var, val in ctr.expr.expr.items():
            a.append((ctr_idx, var.idx, val*mul))
        b.append(mul*ctr.rhs)
        
    for v in mps.vars:
        if v.ub < 1e20:
            a_ineq.append((len(b_ineq), v.idx, 1.0))
            b_ineq.append(v.ub)
        if v.lb > -1e20:
            a_ineq.append((len(b_ineq), v.idx, -1.0))
            b_ineq.append(-v.lb)
            
    c = np.zeros((1, len(mps.vars)))
    for var, val in mps.objective.expr.items():
        c[0,var.idx] = val
        
    csr_a_eq = sp.coo_array(([x[2] for x in a_eq], ([x[0] for x in a_eq], [x[1] for x in a_eq])), shape=(len(b_eq), len(mps.vars))).tocsr()
    csr_a_ineq = sp.coo_array(([x[2] for x in a_ineq], ([x[0] for x in a_ineq], [x[1] for x in a_ineq])), shape=(len(b_ineq), len(mps.vars))).tocsr()
    
    orig_a_eq = csr_a_eq
    orig_a_ineq = csr_a_ineq
    orig_b_eq = np.array(b_eq).reshape((-1, 1))
    orig_b_ineq = np.array(b_ineq).reshape((-1, 1))
    orig_c = c
    
    random = np.random.default_rng(12280752884384100672342137672332424406)
    n_perturbated_ctrs = 100
    min_perturbation_ratio = 0.3
    max_perturbation_ratio = 0.9
    
    # first randomly select len(mps.ctrs)*ratio_ctrs constraints
    n_ctrs = csr_a_eq.shape[0] + csr_a_ineq.shape[0]
    
    FOUND_VALUES = []
    
    for itr in range(1000):
        if itr > 200 and len(FOUND_VALUES):
            break
        #print("START RUN", "CURRENT BEST", 0 if not FOUND_VALUES else FOUND_VALUES[-1][0])
        if MODE == "ALL":
            try:
                perturbated_cstrs = random.choice(n_ctrs, size=n_perturbated_ctrs, replace=False)
                perturbated_cstrs_eq = [x for x in perturbated_cstrs if x < csr_a_eq.shape[0]]
                perturbated_cstrs_ineq = [x-csr_a_eq.shape[0] for x in perturbated_cstrs if x >= csr_a_eq.shape[0]]
            except:
                continue
        elif MODE == "INEQ":
            try:
                perturbated_cstrs_eq = []
                perturbated_cstrs_ineq = random.choice(csr_a_ineq.shape[0], size=n_perturbated_ctrs, replace=False)
            except:
                continue
        elif MODE == "EQ":
            try:
                perturbated_cstrs_ineq = []
                perturbated_cstrs_eq = random.choice(csr_a_eq.shape[0], size=n_perturbated_ctrs, replace=False)
            except:
                continue
        else:
            raise Exception("Unknown mode")
        
        def separate(csr_array, perturbated_idx):
            perturbated_idx_set = set(perturbated_idx)
            not_perturbated_idx = [x for x in range(csr_array.shape[0]) if x not in perturbated_idx_set]
            return csr_array[not_perturbated_idx, :], csr_array[perturbated_idx, :]
        
        a1_eq, a2_eq = separate(orig_a_eq, perturbated_cstrs_eq)
        b1_eq, b2_eq = separate(orig_b_eq, perturbated_cstrs_eq)
        a1_ineq, a2_ineq = separate(orig_a_ineq, perturbated_cstrs_ineq)
        b1_ineq, b2_ineq = separate(orig_b_ineq, perturbated_cstrs_ineq)
    
        def compute_perturbation(csr_array, nb_perturbated_per_row=3):
            data = np.zeros_like(csr_array.data)
            for row in range(csr_array.shape[0]):
                nb = csr_array.indptr[row+1] - csr_array.indptr[row]
                perturbated = random.choice(nb, size=min(nb, nb_perturbated_per_row)) + csr_array.indptr[row]
                data[perturbated] = csr_array.data[perturbated] * random.uniform(min_perturbation_ratio, max_perturbation_ratio, size=min(nb, nb_perturbated_per_row)) * random.choice([-1, 1], size=min(nb, nb_perturbated_per_row))
            return sp.csr_array((data, csr_array.indices.copy(), csr_array.indptr.copy()), shape=csr_array.shape)
        
        d_eq = compute_perturbation(a2_eq)
        d_ineq = compute_perturbation(a2_ineq)
        
        problem = Problem_sparse(
                        a_1_eq=a1_eq,
                        b_1_eq=b1_eq,
                        a_1_ineq=a1_ineq,
                        b_1_ineq=b1_ineq,
                        a_2_eq=a2_eq,
                        b_2_eq=b2_eq,
                        d_eq=d_eq,
                        a_2_ineq=a2_ineq,
                        b_2_ineq=b2_ineq,
                        d_ineq=d_ineq,
                        c=c.transpose(),
                        minimize=mps.sense == "MIN",
                        range=(-1, 1))
        
        out = solve(problem, -1)
        if np.isnan(out):
            #print("SKIP", out)
            continue
        out2 = solve(problem, 0)
        if np.isnan(out2):
            #print("SKIP", out, out2)
            continue
        if np.abs(out-out2) < 1:
            #print("SKIP", out, out2)
            continue
        out3 = solve(problem, 1)
        if np.isnan(out3):
            #print("SKIP", out, out2, out3)
            continue
        if np.abs(out3-out2) < 1:
            #print("SKIP", out, out2, out3)
            continue
        if out <= out2 <= out3 or out3 <= out2 <= out:
            #print("SKIP", out, out2, out3)
            continue
        if np.abs(out-out3) > 1 and (len(FOUND_VALUES) == 0 or FOUND_VALUES[-1][0] < np.abs(out-out3)):
            FOUND_VALUES.append((
                np.abs(out-out3),
                problem
            ))
            #print("FOUND BETTER", np.abs(out-out3), out, out2, out3)
    
    if len(FOUND_VALUES) == 0:
        return None
    
    best_problem = FOUND_VALUES[-1][1]
    
    space = np.linspace(-1, 1, 100)
    start_time = time.time()
    truth(best_problem, [-1, 1])["bounds"][0]["bound"](space)
    end_time = time.time()
    
    rstart_time = time.time()
    robust_concave_envelope(best_problem, [-1, 1])
    rend_time = time.time()
    
    return best_problem, end_time-start_time, rend_time - rstart_time, FOUND_VALUES[-1][0]

def get_folder_name(filename):
    suffix = ""
    if MODE == "INEQ":
        suffix = "__ineq"
    elif MODE == "EQ":
        suffix = "__eq"
    
    name = ".".join(filename.split(".")[:-1])
    f = f"netlib/{name}{suffix}"
    return f

def check_if_done(filename):
    return os.path.exists(get_folder_name(filename))

def store_problem(filename, problem, truth_time, robust_time, delta):
    f = get_folder_name(filename)
    os.mkdir(f)
    sp.save_npz(f"{f}/a_1_eq.npz", problem.a_1_eq)
    sp.save_npz(f"{f}/a_2_eq.npz", problem.a_2_eq)
    sp.save_npz(f"{f}/a_1_ineq.npz", problem.a_1_ineq)
    sp.save_npz(f"{f}/a_2_ineq.npz", problem.a_2_ineq)
    sp.save_npz(f"{f}/d_eq.npz", problem.d_eq)
    sp.save_npz(f"{f}/d_ineq.npz", problem.d_ineq)
    np.savez(f"{f}/b_1_eq.npz", problem.b_1_eq)
    np.savez(f"{f}/b_2_eq.npz", problem.b_2_eq)
    np.savez(f"{f}/b_1_ineq.npz", problem.b_1_ineq)
    np.savez(f"{f}/b_2_ineq.npz", problem.b_2_ineq)
    np.savez(f"{f}/c.npz", problem.c.transpose())
    json.dump({"truth_time": truth_time, "robust_time": robust_time, "delta": delta}, open(f"{f}/data.json", "w"))

In [55]:
for p in files:
    if check_if_done(p):
        continue
    out = gen_random(p)
    print("-------------------")
    print("-------------------")
    print("-------------------")
    if out is None:
        print("NO SOLUTION FOR ", p)
        print("-------------------")
        print("-------------------")
        print("-------------------")
        continue
    best_problem, truth_time, robust_time, delta = out
    print(p, truth_time, robust_time, delta)
    print("-------------------")
    print("-------------------")
    print("-------------------")
    store_problem(p, best_problem, truth_time, robust_time, delta)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/lotfi.mps
Reading time = 0.00 seconds
LOTFI: 153 rows, 308 columns, 1078 nonzeros
-------------------
-------------------
-------------------
NO SOLUTION FOR  lotfi.mps
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/agg3.mps
Reading time = 0.00 seconds
AGG3: 516 rows, 302 columns, 4300 nonzeros
-------------------
-------------------
-------------------
NO SOLUTION FOR  agg3.mps
-------------------
-------------------
-------------------
Set parameter Username
Academ

Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowit

Line(a=array([54404.27089686]), b=array([-4260217.03239542]), limits=(-1, 1))
Line(a=array([25206.0094239]), b=array([-4289291.4231525]), limits=(-1, 1))
Line(a=array([23359.049418]), b=array([-4291086.12840368]), limits=(-1, 1))
Line(a=array([22417.25021027]), b=array([-4291985.29044133]), limits=(-1, 1))
Line(a=array([21360.14903206]), b=array([-4292966.9856286]), limits=(-1, 1))
Line(a=array([20191.84737686]), b=array([-4294035.98689798]), limits=(-1, 1))
Line(a=array([20107.36204017]), b=array([-4294111.68133895]), limits=(-1, 1))
Line(a=array([19993.16128577]), b=array([-4294210.71224573]), limits=(-1, 1))
Line(a=array([19111.9087889]), b=array([-4294961.31328196]), limits=(-1, 1))
Line(a=array([18015.46737477]), b=array([-4295869.81662669]), limits=(-1, 1))
Line(a=array([17781.17540479]), b=array([-4296059.31085022]), limits=(-1, 1))
Line(a=array([17603.13732532]), b=array([-4296199.32685332]), limits=(-1, 1))
Line(a=array([17346.99278325]), b=array([-4296396.94812246]), limits=(

Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1


-------------------
-------------------
-------------------
NO SOLUTION FOR  wood1p.mps
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/d6cube.mps
Reading time = 0.01 seconds
D6CUBE: 415 rows, 6184 columns, 37704 nonzeros
-------------------
-------------------
-------------------
d6cube.mps 11.647840976715088 0.11054277420043945 253.63533636829823
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/sctap1.mps
Reading time = 0.00 seconds
SCTAP1: 300 rows, 480 

...

-------------------
-------------------
-------------------
pilot4.mps 2.666984796524048 0.01636505126953125 1939.678858376229
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/blend.mps
Reading time = 0.00 seconds
BLEND: 74 rows, 83 columns, 491 nonzeros
-------------------
-------------------
-------------------
NO SOLUTION FOR  blend.mps
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/degen2.mps
Reading time = 0.00 seconds
DEGEN2: 444 rows, 534 columns, 3

Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1


-------------------
-------------------
-------------------
degen2.mps 2.145829200744629 0.014966964721679688 126.63272743664993
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/degen3.mps
Reading time = 0.00 seconds
DEGEN3: 1503 rows, 1818 columns, 24646 nonzeros


Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1
Markowitz threshold set to 0.1


-------------------
-------------------
-------------------
degen3.mps 12.534847974777222 0.10055112838745117 4.0392016184382555
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/share1b.mps
Reading time = 0.00 seconds
SHARE1B: 117 rows, 225 columns, 1151 nonzeros
-------------------
-------------------
-------------------
NO SOLUTION FOR  share1b.mps
-------------------
-------------------
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Read MPS format model from file /Users/gderval/Downloads/Data-Netlib-master/woodw.mps
Reading time = 0.01 seconds
WOODW: 1098 rows, 8405